In [ ]:
import kfp.dsl as dsl
from string import Template
import json

In [ ]:
tfjobjson_template = Template("""
{
    "apiVersion": "kubeflow.org/v1",
    "kind": "TFJob",
    "metadata": {
        "name": "tfjob-custom-code-{{workflow.uid}}",
        "namespace": "dejan-golubovic"
    },
    "spec": {
        "tfReplicaSpecs": {
            "Worker": {
                "replicas": 1,
                "template": {
                    "spec": {
                        "containers": [
                            {
                                "image": "registry.hub.docker.com/dejangolubovic/tfjob-test:customcode4",
                                "name": "tensorflow",
                                "command": ["python", "/custom-code.py"]
                            }
                        ],
                        "restartPolicy": "Never"
                    },
                    "metadata": {
                        "annotations": {
                            "sidecar.istio.io/inject": "false"
                        }
                    }
                }
            }
        }
    }
}
""")

tfjobjson = tfjobjson_template.substitute()
tfjob = json.loads(tfjobjson)

In [ ]:
def tfjob_pipeline():
    tfjob_op = dsl.ResourceOp(
        name='test_tf_job',
        k8s_resource=tfjob,
        success_condition='status.replicaStatuses.Worker.succeeded == 1'
    )

In [ ]:
import kfp.compiler as compiler
compiler.Compiler().compile(tfjob_pipeline, "tfjob_pip.yaml")